In [1]:
#Main Function
import requests
from bs4 import BeautifulSoup
import time
import threading
import urllib.request #Save Image
import webbrowser #멮쥐
#File
from datetime import datetime
import os
from PIL import ImageTk
import PIL.Image #ImageTk와 네임스페이스 충돌하여 따로 설정
#UI
from tkinter import *
from tkinter import ttk
import tkinter.messagebox

In [3]:
class UiText():
    def ResetStatusLabels():
        label_information.config(text ='')
        UiText.ProgressText('0.0%')
        UiText.SuspectAmount(0)
        UiText.ExpectedTime(0)
                
    def ActivateButton(new_text):
        button_activate.config(text = new_text)
    
    def CalculateProgressPercent(char_num, present_page, page_min, page_max):
        progress_total = (page_max-page_min+1)*10
        progress_present = (present_page-page_min)*10+(char_num+1)
        percent = round((progress_present/progress_total)*100,1)
        label_progress.config(text = ( str(percent)+'%' ))
        
    def ProgressText(new_text):
        label_progress.config(text = new_text)
        
    def SuspectAmount(amount):
        label_suspect_amount.config(text = '['+ str(amount)+ '명]')
        
    def ExpectedTime(time):
        if time <= 0:
            label_expect_time.config(text = '')
        else:
            label_expect_time.config(text = '예상 시간 : '+str(round(time,2))+ '초')
            
    def InformationLabel(new_text):
        label_information.config(text = new_text)
        
    def ClearEntryText(*clear_entrys):
        for clear_entry in clear_entrys:
            if clear_entry == 'union':
                entry_union_min.delete(0, len(entry_union_min.get()))
                entry_union_max.delete(0, len(entry_union_max.get()))
            if clear_entry == 'achieve':    
                entry_achieve_min.delete(0, len(entry_achieve_min.get()))
                entry_achieve_max.delete(0, len(entry_achieve_max.get()))
            if clear_entry == 'ingido':
                entry_ingido_min.delete(0, len(entry_ingido_min.get()))
                entry_ingido_max.delete(0, len(entry_ingido_max.get()))
            if clear_entry == 'level' or clear_entry == 'page':
                entry_level_min.delete(0, len(entry_level_min.get()))
                entry_level_max.delete(0, len(entry_level_max.get()))
            
def ShowJobList(event):
    job_category = combobox_job_category.get()
    combobox_job.set('')
    
    if job_category == '[모전]' : job_list = ['히어로', '팔라딘', '다크나이트']
    elif job_category == '[모법]' : job_list = ['불독', '썬콜', '비숍']
    elif job_category == '[모궁]' : job_list = ['보우마스터', '신궁', '패스파인더']
    elif job_category == '[모도]' : job_list = ['나이트로드', '섀도어', '듀얼블레이더']
    elif job_category == '[모해]' : job_list = ['바이퍼', '캡틴', '캐논슈터']    
    elif job_category == '[시그너스]' : job_list = ['소울마스터', '플레임위자드', '윈드브레이커', '나이트워커', '스트라이커', '미하일'] 
    elif job_category == '[레지스탕스]' : job_list = ['배틀메이지', '와일드헌터', '메카닉','데몬슬레이어', '데몬어벤져', '제논', '블래스터'] 
    elif job_category == '[영웅]': job_list = ['아란', '에반', '메르세데스', '팬텀', '루미너스', '은월'] 
    elif job_category == '[노바]': job_list = ['카이저', '엔버', '카데나', '카인'] 
    elif job_category == '[레프]': job_list = ['아크', '일리움', '아델'] 
    elif job_category == '[아니마]': job_list = ['호영', '라라']     
    elif job_category == '[혼밥족]': job_list = ['키네시스', '제로'] 
    
    combobox_job["state"] = "readonly"
    combobox_job["values"] = job_list

def EnableFindAllCharacter():
    if is_checked_find_all.get() == True:
        tkinter.messagebox.showinfo("Message", "지정한 레벨/페이지 내 모든 캐릭터의 정보를 저장합니다.\n (엑셀로 필터링 할 경우 권장합니다.)") 
        UiText.ClearEntryText('union', 'achieve', 'ingido')
        entry_union_min.insert(0, '0')
        entry_union_max.insert(0, '100000')
        entry_achieve_min.insert(0, '0')
        entry_achieve_max.insert(0, '100000')
        entry_ingido_min.insert(0, '-100000')
        entry_ingido_max.insert(0, '100000')
        entry_union_min['state'] = 'disable'
        entry_union_max['state'] = 'disable'
        entry_achieve_min['state'] = 'disable'
        entry_achieve_max['state'] = 'disable'
        entry_ingido_min['state'] = 'disable'
        entry_ingido_max['state'] = 'disable'
    else:
        entry_union_min['state'] = 'normal'
        entry_union_max['state'] = 'normal'
        entry_achieve_min['state'] = 'normal'
        entry_achieve_max['state'] = 'normal'
        entry_ingido_min['state'] = 'normal'
        entry_ingido_max['state'] = 'normal'        
        UiText.ClearEntryText('union', 'achieve', 'ingido')
    
def EnableSelfSetPage():
    if is_checked_self_setpage.get() == True:
        tkinter.messagebox.showinfo("Message", "레벨 범위 설정이 페이지 범위 설정으로 변경됩니다.\n(홈페이지에서 검색할 페이지의 범위를 입력해주세요)") 
        UiText.ClearEntryText('level')
        label_level.config(text = '[페이지 범위]')
        label_level.config(foreground = 'blue')
        label_level.place(x = 93, y = 250)
        
        global g_page_min, g_page_max
        if g_page_min != 0 and g_page_max != 0 and tkinter.messagebox.askquestion('Message','이전에 검색한 페이지 범위를 입력하시겠습니까?\n'+ str(g_page_min) + ' ~ ' + str(g_page_max) + ' 페이지') == 'yes':
            entry_level_min.insert(0, g_page_min)
            entry_level_max.insert(0, g_page_max)  
    else:
        label_level.config(text = '[레벨 범위]')
        label_level.config(foreground = 'black')
        label_level.place(x = 100, y = 250)        
        UiText.ClearEntryText('level')
        
def AchieveCheckBtnMessage():
    if is_checked_no_achieve_btn.get() == True:
        tkinter.messagebox.showinfo("Message", "기능이 활성화되어 업적이 없는 캐릭터도 검색합니다.") 

#[Set Windows]
win = Tk()
win.title('추노')    
win.geometry('430x345')
win.option_add('*Font', '맑은고딕 10')
win.resizable(False, False)

#[Set Image]
label_image = Label(win)
main_img_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
main_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_img_url, stream=True).raw))
label_image.config(image = main_img)
label_image.pack()

#[Set Server]
label_server = Label(win, text = '[서버 설정]')

server_list =['전체월드', '루나', '스카니아', '엘리시움', '크로아', '베라', 
              '오로라', '레드', '제니스', '이노시스', '유니온', 
              '아케인', '노바', '리부트1', '리부트2'] 

combobox_server = ttk.Combobox(win, 
                               width = 10, 
                               values = server_list, 
                               state="readonly")
combobox_server.set("서버 선택")
label_server.place(x = 100 , y = 100)
combobox_server.place(x = 200, y = 100)

#[Set Job]
label_job = Label(win, text = '[직업 설정]')

job_category =['[모전]', '[모법]', '[모궁]', '[모도]', '[모해]',
               '[시그너스]',
               '[영웅]',
               '[레지스탕스]',
               '[노바]',
               '[레프]',
               '[아니마]',
               '[혼밥족]'
               ] 
combobox_job_category = ttk.Combobox(win, 
                                     width = 10, 
                                     values = job_category, 
                                     state="readonly")
    #job_category Choice Event
combobox_job_category.bind("<<ComboboxSelected>>", ShowJobList)
    #job_category Text
combobox_job_category.set("직업군 선택")

combobox_job = ttk.Combobox(win, width = 10, state="disabled")

label_job.place(x = 100 , y = 130)
combobox_job_category.place(x = 200, y = 130)    
combobox_job.place(x = 300, y = 130)    

#[Set Union Score]
label_union = Label(win, text = '[유니온 범위]')

entry_union_min = Entry(win, width = 5)
label_union_range = Label(win, text = '-')
entry_union_max = Entry(win, width = 5)

label_union.place(x = 93, y = 160)
entry_union_min.place(x = 200, y = 160)
entry_union_max.place(x = 260, y = 160)
label_union_range.place(x = 240, y = 160)

#[Set Achievement Score]
label_achieve = Label(win, text = '[업적 범위]')

entry_achieve_min = Entry(win, width = 5)
label_achieve_range = Label(win, text = '-')
entry_achieve_max = Entry(win, width = 5)

label_achieve.place(x = 100, y = 190)
entry_achieve_min.place(x = 200, y = 190)
entry_achieve_max.place(x = 260, y = 190)
label_achieve_range.place(x = 240, y = 190)  

#[Set Ingido]
label_ingido = Label(win, text = '[인기도 범위]')

entry_ingido_min = Entry(win, width = 5)
label_ingido_range = Label(win, text = '-')
entry_ingido_max = Entry(win, width = 5)

label_ingido.place(x = 93, y = 220)
entry_ingido_min.place(x = 200, y = 220)
entry_ingido_max.place(x = 260, y = 220)
label_ingido_range.place(x = 240, y = 220)

#[Set Level]
label_level = Label(win, text = '[레벨 범위]')

entry_level_min = Entry(win, width = 5)
label_level_range = Label(win, text = '-')
entry_level_max = Entry(win, width = 5)

label_level.place(x = 100, y = 250)
entry_level_min.place(x = 200, y = 250)
entry_level_max.place(x = 260, y = 250)
label_level_range.place(x = 240, y = 250)

#[Progress Label]
label_progress = Label(win)
label_progress.place(x = 195 , y = 273)

label_suspect_amount = Label(win)
label_suspect_amount.place(x = 235 , y = 273)

label_expect_time = Label(win)
label_expect_time.place(x = 290 , y = 273)

label_information = Label(win, font = '맑은고딕 9')
label_information.place(x = 130, y = 273)

#[Option Check Button]
is_checked_self_setpage = BooleanVar()
chbtn_self_setpage = Checkbutton(win,
                    text = "검색 페이지 직접 설정",
                    variable = is_checked_self_setpage,
                    font = '맑은고딕 8')
chbtn_self_setpage.config(command = EnableSelfSetPage)
chbtn_self_setpage.place(x = 290, y = 291) 

#[Activate Button]
button_activate = Button(win, 
                         width = 20,
                         height = 2,
                         text = 'Activate')
button_activate.place(x = 120 , y = 298)

#Start GUI
win.mainloop()

In [2]:
class UiText():
    def ResetStatusLabels():
        label_information.config(text ='')
        UiText.ProgressText('0.0%')
        UiText.SuspectAmount(0)
        UiText.ExpectedTime(0)
                
    def ActivateButton(new_text):
        button_activate.config(text = new_text)
    
    def CalculateProgressPercent(char_num, present_page, page_min, page_max):
        progress_total = (page_max-page_min+1)*10
        progress_present = (present_page-page_min)*10+(char_num+1)
        percent = round((progress_present/progress_total)*100,1)
        label_progress.config(text = ( str(percent)+'%' ))
        
    def ProgressText(new_text):
        label_progress.config(text = new_text)
        
    def SuspectAmount(amount):
        label_suspect_amount.config(text = '['+ str(amount)+ '명]')
        
    def ExpectedTime(time):
        if time <= 0:
            label_expect_time.config(text = '')
        else:
            label_expect_time.config(text = '예상 시간 : '+str(round(time,2))+ '초')
            
    def InformationLabel(new_text):
        label_information.config(text = new_text)
        
    def ClearEntryText(*clear_entrys):
        for clear_entry in clear_entrys:
            if clear_entry == 'union':
                GUI.entry_union_min.delete(0, len(GUI.entry_union_min.get()))
                GUI.entry_union_max.delete(0, len(entry_union_max.get()))
            if clear_entry == 'achieve':    
                GUI.entry_achieve_min.delete(0, len(GUI.entry_achieve_min.get()))
                GUI.entry_achieve_max.delete(0, len(GUI.entry_achieve_max.get()))
            if clear_entry == 'ingido':
                GUI.entry_ingido_min.delete(0, len(GUI.entry_ingido_min.get()))
                GUI.entry_ingido_max.delete(0, len(GUI.entry_ingido_max.get()))
            if clear_entry == 'level' or clear_entry == 'page':
                GUI.entry_level_min.delete(0, len(GUI.entry_level_min.get()))
                GUI.entry_level_max.delete(0, len(GUI.entry_level_max.get()))

def ShowJobList(event):
    job_category = GUI.combobox_job_category.get()
    GUI.combobox_job.set('')
    
    if job_category == '[모전]' : job_list = ['히어로', '팔라딘', '다크나이트']
    elif job_category == '[모법]' : job_list = ['불독', '썬콜', '비숍']
    elif job_category == '[모궁]' : job_list = ['보우마스터', '신궁', '패스파인더']
    elif job_category == '[모도]' : job_list = ['나이트로드', '섀도어', '듀얼블레이더']
    elif job_category == '[모해]' : job_list = ['바이퍼', '캡틴', '캐논슈터']    
    elif job_category == '[시그너스]' : job_list = ['소울마스터', '플레임위자드', '윈드브레이커', '나이트워커', '스트라이커', '미하일'] 
    elif job_category == '[레지스탕스]' : job_list = ['배틀메이지', '와일드헌터', '메카닉','데몬슬레이어', '데몬어벤져', '제논', '블래스터'] 
    elif job_category == '[영웅]': job_list = ['아란', '에반', '메르세데스', '팬텀', '루미너스', '은월'] 
    elif job_category == '[노바]': job_list = ['카이저', '엔버', '카데나', '카인'] 
    elif job_category == '[레프]': job_list = ['아크', '일리움', '아델'] 
    elif job_category == '[아니마]': job_list = ['호영', '라라']     
    elif job_category == '[혼밥족]': job_list = ['키네시스', '제로'] 
    
    GUI.combobox_job["state"] = "readonly"
    GUI.combobox_job["values"] = job_list

def EnableSelfSetPage():
    if GUI.is_checked_self_setpage.get() == True:
        tkinter.messagebox.showinfo("Message", "레벨 범위 설정이 페이지 범위 설정으로 변경됩니다.\n(홈페이지에서 검색할 페이지의 범위를 입력해주세요)") 
        UiText.ClearEntryText('level')
        GUI.label_level.config(text = '[페이지 범위]')
        GUI.label_level.config(foreground = 'blue')
        GUI.label_level.place(x = 93, y = 210)
        
#         global g_page_min, g_page_max
#         if g_page_min != 0 and g_page_max != 0 and tkinter.messagebox.askquestion('Message','이전에 검색한 페이지 범위를 입력하시겠습니까?\n'+ str(g_page_min) + ' ~ ' + str(g_page_max) + ' 페이지') == 'yes':
#             GUI.entry_level_min.insert(0, g_page_min)
#             GUI.entry_level_max.insert(0, g_page_max)  
    else:
        GUI.label_level.config(text = '[레벨 범위]')
        GUI.label_level.config(foreground = 'black')
        GUI.label_level.place(x = 100, y = 210)        
        UiText.ClearEntryText('level')

In [13]:
class UiMain():
    def __init__(self):
        self.win = Tk()
        self.win.title('추노')    
        self.win.geometry('430x315')
        self.win.option_add('*Font', '맑은고딕 10')
        self.win.resizable(False, False)
        #Mode Select[Radio Button]
        self.radio_variety=tkinter.IntVar()
        self.mode_select_1 = Radiobutton(self.win, text="캐릭터 정보 수집 모드", font = '맑은고딕 8', value=1, variable=self.radio_variety, command = lambda:UiMain.GetCharacterInfoModeUI(self))
        self.mode_select_1.select()
        self.mode_select_2 = Radiobutton(self.win, text="정보 필터링 모드", font = '맑은고딕 8', value=2, variable=self.radio_variety, command = lambda:UiMain.FilterCharacterInfoModeUI(self)) 
        self.mode_select_1.place(x = 70, y = 90)
        self.mode_select_2.place(x = 250, y = 90)
        #Default Ui
        UiMain.MainImage(self)
        UiMain.GetCharacterInfoModeUI(self)
        
    def MainImage(self):
        self.label_image = Label(self.win)
        main_img_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
        main_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_img_url, stream=True).raw))
        self.label_image.config(image = main_img)
        self.label_image.image = main_img # 파이썬 가비지 컬렉션의 이미지 삭제 방지
        self.label_image.pack()
        
    def MakeGetInfoModeUi():
        
    def HideGetInfoModeUi(self):
        self.label_server.place_forget()
        self.label_job.place_forget()
        self.label_level.place_forget()
        self.label_level_range.place_forget()
        
        self.combobox_server.place_forget()
        self.combobox_job_category.place_forget()
        self.combobox_job.place_forget()
        
        self.entry_level_min.place_forget()
        self.entry_level_max.place_forget()
        
        self.chbtn_self_setpage.place_forget()
        
    def HideFilterModeUi(self):
        self.label_union.place_forget()
        self.label_union_range.place_forget()
        self.label_achieve.place_forget()
        self.label_achieve_range.place_forget()
        self.label_ingido.place_forget()
        self.label_ingido_range.place_forget()
        
        self.entry_union_min.place_forget()
        self.entry_union_max.place_forget()
        self.entry_achieve_min.place_forget()
        self.entry_achieve_max.place_forget()
        self.entry_ingido_min.place_forget()
        self.entry_ingido_max.place_forget()
        
    def GetCharacterInfoModeUI(self):
        #이미 GetInfo 모드인 상태에서 GetInfo 버튼을 누를 경우 삭제
        try : UiMain.HideGetInfoModeUi(self)
        except: pass         
        
        #Filter Ui 삭제
        try: UiMain.HideFilterModeUi(self)
        except: pass #이미 열려있거나 forget 상태여서 Attribute Error 뜨면 패스
        
        #[Set Server]
        self.label_server = Label(self.win, text = '[서버 설정]')

        self.server_list =['전체월드', '루나', '스카니아', '엘리시움', '크로아', '베라', 
                      '오로라', '레드', '제니스', '이노시스', '유니온', 
                      '아케인', '노바', '리부트1', '리부트2'] 

        self.combobox_server = ttk.Combobox(self.win, 
                                       width = 10, 
                                       values = self.server_list, 
                                       state="readonly")
        self.combobox_server.set("서버 선택")
        self.label_server.place(x = 100 , y = 150)
        self.combobox_server.place(x = 200, y = 150)        
        
        #[Set Job]
        self.label_job = Label(self.win, text = '[직업 설정]')

        self.job_category =['[모전]', '[모법]', '[모궁]', '[모도]', '[모해]',
                       '[시그너스]',
                       '[영웅]',
                       '[레지스탕스]',
                       '[노바]',
                       '[레프]',
                       '[아니마]',
                       '[혼밥족]'
                       ] 
        self.combobox_job_category = ttk.Combobox(self.win, 
                                             width = 10, 
                                             values = self.job_category, 
                                             state="readonly")
            #job_category Choice Event
        self.combobox_job_category.bind("<<ComboboxSelected>>", ShowJobList)
            #job_category Text
        self.combobox_job_category.set("직업군 선택")

        self.combobox_job = ttk.Combobox(self.win, width = 10, state="disabled")

        self.label_job.place(x = 100 , y = 180)
        self.combobox_job_category.place(x = 200, y = 180)    
        self.combobox_job.place(x = 300, y = 180)            
        #[Set Level]
        self.label_level = Label(self.win, text = '[레벨 범위]')

#         self.entry_level_min = Entry(self.win, width = 5)
        self.label_level_range = Label(self.win, text = '-')
        self.entry_level_max = Entry(self.win, width = 5)

        self.label_level.place(x = 100, y = 210)
        self.entry_level_min.place(x = 200, y = 210)
        self.entry_level_max.place(x = 260, y = 210)
        self.label_level_range.place(x = 240, y = 210)   
        
        #[Progress Label]
        self.label_progress = Label(self.win)
        self.label_progress.place(x = 195 , y = 273)

        self.label_suspect_amount = Label(self.win)
        self.label_suspect_amount.place(x = 235 , y = 273)

        self.label_expect_time = Label(self.win)
        self.label_expect_time.place(x = 290 , y = 273)

        self.label_information = Label(self.win, font = '맑은고딕 9')
        self.label_information.place(x = 130, y = 273)

        #[Option Check Button]
        self.is_checked_self_setpage = BooleanVar()
        self.chbtn_self_setpage = Checkbutton(self.win,
                            text = "검색 페이지 직접 설정",
                            variable = self.is_checked_self_setpage,
                            font = '맑은고딕 8')
        self.chbtn_self_setpage.config(command = EnableSelfSetPage)
        self.chbtn_self_setpage.place(x = 290, y = 291)         

        
        #[Activate Button]
        self.button_activate = Button(self.win, 
                                 width = 20,
                                 height = 2,
                                 text = 'Activate')
        self.button_activate.place(x = 120 , y = 268)     
        
    def FilterCharacterInfoModeUI(self):
        #이미 Filter 모드인 상태에서 Filter 버튼을 누를 경우 삭제
        try : UiMain.HideFilterModeUi(self)
        except: pass 
        
        #기존 GetInfo Ui 삭제
        try : UiMain.HideGetInfoModeUi(self)
        except: pass #이미 열려있거나 forget 상태여서 Attribute Error 뜨는 경우
        
        print(self.entry_level_min.get())
        
        #[Set Union Score]
        self.label_union = Label(self.win, text = '[유니온 범위]')

        self.entry_union_min = Entry(self.win, width = 5)
        self.label_union_range = Label(self.win, text = '-')
        self.entry_union_max = Entry(self.win, width = 5)

        self.label_union.place(x = 93, y = 160)
        self.entry_union_min.place(x = 200, y = 160)
        self.label_union_range.place(x = 240, y = 160)
        self.entry_union_max.place(x = 260, y = 160)

        #[Set Achievement Score]
        self.label_achieve = Label(self.win, text = '[업적 범위]')

        self.entry_achieve_min = Entry(self.win, width = 5)
        self.label_achieve_range = Label(self.win, text = '-')
        self.entry_achieve_max = Entry(self.win, width = 5)

        self.label_achieve.place(x = 100, y = 190)
        self.entry_achieve_min.place(x = 200, y = 190)
        self.entry_achieve_max.place(x = 260, y = 190)
        self.label_achieve_range.place(x = 240, y = 190) 

        #[Set Ingido]
        self.label_ingido = Label(self.win, text = '[인기도 범위]')

        self.entry_ingido_min = Entry(self.win, width = 5)
        self.label_ingido_range = Label(self.win, text = '-')
        self.entry_ingido_max = Entry(self.win, width = 5)

        self.label_ingido.place(x = 93, y = 220)
        self.entry_ingido_min.place(x = 200, y = 220)
        self.entry_ingido_max.place(x = 260, y = 220)
        self.label_ingido_range.place(x = 240, y = 220)            
GUI = UiMain()
GUI.win.mainloop()

asd
asd
asd
